# Phase 2 : Analyse des données

L'objectif dans cette phase est d'Analyser les données de MovieLens via le SDK et en déduire des tendances et statistiques clés.

In [1]:
from trinite_movies_sdk import MovieClient, MovieConfig

/home/trinite/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


# Se familiariser à l'API

In [2]:

# Configuration avec l’URL de votre API (Render ou locale)
config = MovieConfig(movie_base_url="https://movie-backend-yvle.onrender.com")
client = MovieClient(config=config)

MOVIE_API_BASE_URL in MovieConfig init: https://movie-backend-yvle.onrender.com


In [3]:
client.health_check()

{'message': 'Api Movie opérationnelle'}

In [4]:
# Récupération d'un film pour test
movie=client.get_movie(1)
print(f"Titre: {movie.title}")
print(f"Genres: {movie.genres}")

Titre: Toy Story (1995)
Genres: Adventure|Animation|Children|Comedy|Fantasy


In [6]:
# On récupère une grande portion de la table ( à ajuster selon la taille réelle)
ratings_df=client.list_ratings(limit=100, output_format="pandas")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     100 non-null    int64  
 1   movieId    100 non-null    int64  
 2   rating     100 non-null    float64
 3   timestamp  100 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 3.2 KB


In [8]:
# Récupération des statistiques globales
analytics= client.get_analytics()
print(analytics)

movie_count=9742 rating_count=100836 tag_count=3683 link_count=9742


In [10]:
total_rating=analytics.rating_count
total_rating

100836

In [11]:
complete_rating_df=client.list_ratings(limit=total_rating,output_format="pandas")
complete_rating_df.head()

HTTPStatusError: Client error '422 Unprocessable Entity' for url 'https://movie-backend-yvle.onrender.com/ratings?skip=0&limit=100836'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/422

L'erreur signifie que l'API **refuse de traiter ta requête** parce que le paramètre `limit=100836` est trop élevé pour elle. En général, les APIs REST ont une **limite maximale de résultats** que tu peux demander en une seule fois — souvent autour de 100, 500, voire 1000. Cela permet non seulement de préserver les performances de l'API et d'éviter des surcharges côté serveur, mais aussi d'assurer un contrôle sur les accès aux données. C'est là l'un des grands avantages de la mise en place d'APIs : elles permettent de **partager des données de manière sécurisée** et d’**encadrer l’utilisation des ressources**.

Ces limites peuvent aussi être mises en place pour des raisons **commerciales**, notamment si l'API est utilisée pour vendre ou monétiser des données. En fixant des limites sur le nombre de résultats ou le nombre d'appels, l'entreprise qui fournit l'API peut contrôler l'accès aux données et éviter des abus tout en gérant ses coûts opérationnels.

Cela permet donc de voir les limites de l'API non seulement sous un angle technique et sécuritaire, mais aussi stratégique et commercial.

In [12]:
import time

In [13]:
import pandas as pd

In [14]:
# 1. récupérer le total d'évaluations
total_ratings=client.get_analytics().rating_count
batch_size=1000
all_ratings=[]

# 2. Boucle surles batches avec pause
for skip in range(0,total_rating, batch_size):
    print(f"Téléchargement des lignes {skip} à {skip + batch_size}...")
    batch_df=client.list_ratings(skip=skip,limit=batch_size, output_format="pandas")
    all_ratings.append(batch_df)
    time.sleep(0.5) # pause de 0.5 seconde

# 3. concaténer tous les résultats
complete_rating_df=pd.concat(all_ratings,ignore_index=True)
complete_rating_df

Téléchargement des lignes 0 à 1000...
Téléchargement des lignes 1000 à 2000...
Téléchargement des lignes 2000 à 3000...
Téléchargement des lignes 3000 à 4000...
Téléchargement des lignes 4000 à 5000...
Téléchargement des lignes 5000 à 6000...
Téléchargement des lignes 6000 à 7000...
Téléchargement des lignes 7000 à 8000...
Téléchargement des lignes 8000 à 9000...
Téléchargement des lignes 9000 à 10000...
Téléchargement des lignes 10000 à 11000...
Téléchargement des lignes 11000 à 12000...
Téléchargement des lignes 12000 à 13000...
Téléchargement des lignes 13000 à 14000...
Téléchargement des lignes 14000 à 15000...
Téléchargement des lignes 15000 à 16000...
Téléchargement des lignes 16000 à 17000...
Téléchargement des lignes 17000 à 18000...
Téléchargement des lignes 18000 à 19000...
Téléchargement des lignes 19000 à 20000...
Téléchargement des lignes 20000 à 21000...
Téléchargement des lignes 21000 à 22000...
Téléchargement des lignes 22000 à 23000...
Téléchargement des lignes 23000 à

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [15]:
complete_rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [22]:
# 4. agrégation :nombre d'évaluation par utilisateur
ratings_per_user = complete_rating_df['userId'].value_counts().rename_axis('userId').reset_index(name='rating_count')

# 5. affichage
ratings_per_user

,userId,rating_count
0,414,2698
1,599,2478
2,474,2108
3,448,1864
4,274,1346
...,...,...
605,442,20
606,569,20
607,320,20
608,576,20


# Question Business pertinente

**Quels genres de films les utilisateurs taguent le plus positivement (note ≥ 4.0), et quels sont les tags les plus fréquents associés à ces genres ?**

### Pourquoi c’est pertinent ?
- Cela permet de **comprendre les préférences des utilisateurs** non seulement à travers les notes, mais aussi via les **tags qualitatifs** qu’ils ajoutent.
- Un **analyste marketing** ou un **algorithme de recommandation** peut utiliser cette information pour :
  - recommander des films similaires,
  - optimiser la classification des films,
  - mieux comprendre les "moods" ou intentions derrière les notes élevées.

---

### Données nécessaires (via SDK + API) :
- **`ratings`** : pour filtrer sur les évaluations élevées (`rating ≥ 4.0`).
- **`tags`** : pour voir quels tags sont utilisés sur les mêmes `(userId, movieId)`.
- **`movies`** : pour enrichir avec les genres correspondants.

---

### Étapes principales :
1. Lister toutes les **ratings ≥ 4.0** (en batch si nécessaire).
2. Pour chaque `(user_id, movieId)` filtré, essayer de récupérer un **tag** via `client.get_tag(...)` ou en listant tous les tags et croisant.
3. Récupérer les **genres du film** via `client.get_movie(movieId)`.
4. Agréger : **genre ↔️ tag ↔️ fréquence**.

In [23]:
analytics

AnalyticsResponse(movie_count=9742, rating_count=100836, tag_count=3683, link_count=9742)

In [25]:
# Étape 1 : Récupérer les évaluations élevées (rating >= 4.0) par lots

chunk_size = 1000
skip = 0
all_high_ratings = []

while True:
    chunk = client.list_ratings(
        skip=skip,
        limit=chunk_size,
        min_rating=4.0,
        output_format="pandas"
    )
    
    if chunk.empty:
        break
    
    all_high_ratings.append(chunk)
    skip += chunk_size
    time.sleep(0.5)  # Pause entre appels pour éviter les erreurs 429

# Fusionner tous les chunks
high_ratings_df = pd.concat(all_high_ratings, ignore_index=True)
print(high_ratings_df.shape)
high_ratings_df

(48580, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
48575,610,166528,4.0,1493879365
48576,610,166534,4.0,1493848402
48577,610,168248,5.0,1493850091
48578,610,168250,5.0,1494273047


In [26]:
# Etape 2 : Identifier les couples (userId, movieId) uniques

user_movie_pairs=high_ratings_df[['userId','movieId']].drop_duplicates()
user_movie_pairs

,userId,movieId
0,1,1
1,1,3
2,1,6
3,1,47
4,1,50
...,...,...
48575,610,166528
48576,610,166534
48577,610,168248
48578,610,168250


In [28]:
# Etape 3 : Récuperer les tags correspondants

# Récupération de tous les tags
all_tags= []
skip = 0
chunk_size = 1000
while True:
    tag_chunk=client.list_tags(skip=skip,limit=chunk_size, output_format="pandas")
    if tag_chunk.empty:
        break
    all_tags.append(tag_chunk)
    skip +=chunk_size
    time.sleep(0.5)
all_tags_df=pd.concat(all_tags,ignore_index=True)
all_tags_df

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [29]:
# Merge avec les high ratings
tagged_high_ratings=pd.merge(user_movie_pairs, all_tags_df,on=["userId","movieId"])
print(tagged_high_ratings.shape)
tagged_high_ratings

(2378, 4)


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
2373,606,6107,World War II,1178473747
2374,606,7382,for katie,1171234019
2375,610,3265,gun fu,1493843984
2376,610,3265,heroic bloodshed,1493843978


In [38]:
# Etape4 : Récuperer lesgenres associésaux movieId

def get_movie_genre(movie_id):
    try:
        movie=client.get_movie(movie_id)
        return movie.genres
    except:
        return ""

# Apliquer uniquement aux movieId unique qu'on a en tags
unique_movie_ids=tagged_high_ratings['movieId'].unique()

movie_genres={
    movie_id:get_movie_genre(movie_id)
    for movie_id in unique_movie_ids
}

# Ajoutons la colonne genre
tagged_high_ratings['genres']= tagged_high_ratings['movieId'].map(movie_genres)
print(tagged_high_ratings.shape)
tagged_high_ratings

(2378, 6)


,userId,movieId,tag,timestamp,genres,genre
0,2,60756,funny,1445714994,Comedy,[Comedy]
1,2,60756,Highly quotable,1445714996,Comedy,[Comedy]
2,2,60756,will ferrell,1445714992,Comedy,[Comedy]
3,2,89774,Boxing story,1445715207,Drama,[Drama]
4,2,89774,MMA,1445715200,Drama,[Drama]
...,...,...,...,...,...,...
2373,606,6107,World War II,1178473747,Drama|War,"[Drama, War]"
2374,606,7382,for katie,1171234019,Drama|Mystery|Thriller,"[Drama, Mystery, Thriller]"
2375,610,3265,gun fu,1493843984,Action|Crime|Drama|Thriller,"[Action, Crime, Drama, Thriller]"
2376,610,3265,heroic bloodshed,1493843978,Action|Crime|Drama|Thriller,"[Action, Crime, Drama, Thriller]"


In [39]:
# Étape 5 : Agrégation finale : genre ↔ tag ↔ count

# On "explose" les genres s'ils sont séparés par "|"
tagged_high_ratings['genres'] = tagged_high_ratings['genres'].str.split('|')
tagged_exploded = tagged_high_ratings.explode('genres')

tagged_exploded

,userId,movieId,tag,timestamp,genres,genre
0,2,60756,funny,1445714994,Comedy,[Comedy]
1,2,60756,Highly quotable,1445714996,Comedy,[Comedy]
2,2,60756,will ferrell,1445714992,Comedy,[Comedy]
3,2,89774,Boxing story,1445715207,Drama,[Drama]
4,2,89774,MMA,1445715200,Drama,[Drama]
...,...,...,...,...,...,...
2376,610,3265,heroic bloodshed,1493843978,Drama,"[Action, Crime, Drama, Thriller]"
2376,610,3265,heroic bloodshed,1493843978,Thriller,"[Action, Crime, Drama, Thriller]"
2377,610,168248,Heroic Bloodshed,1493844270,Action,"[Action, Crime, Thriller]"
2377,610,168248,Heroic Bloodshed,1493844270,Crime,"[Action, Crime, Thriller]"


In [40]:
# Compter les combinaisons Genre / Tag
genre_tag_summary = (
    tagged_exploded
    .groupby(['genres', 'tag'])
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)

genre_tag_summary

,genres,tag,count
1971,Drama,In Netflix queue,20
2159,Drama,atmospheric,19
4321,Thriller,twist ending,16
3280,Mystery,twist ending,14
4304,Thriller,suspense,14
...,...,...,...
1667,Crime,martial arts,1
1668,Crime,masterpiece,1
1669,Crime,meaningless violence,1
1670,Crime,men in drag,1


Ce tableau `genre_tag_summary` fournit une **analyse croisée entre les genres de films et les tags les plus utilisés** par les utilisateurs qui ont **attribué une note élevée** (`rating >= 4.0`). Voici quelques commentaires et interprétations intéressantes :

---

### **Ce que le tableau montre**
- Chaque ligne représente une combinaison unique de **genre** et de **tag**.
- La colonne `count` indique **le nombre de fois** qu’un certain **tag** a été associé à un film d’un certain **genre**, dans le contexte d’une **note élevée**.
- Exemple :  
  - `Drama` + `In Netflix queue` a été tagué **20 fois** pour des films bien notés de genre `Drama`.
  - `Thriller` + `twist ending` a été tagué **16 fois**, ce qui donne des indices sur ce que les gens aiment dans les thrillers.

---

### **Interprétations business**
1. **Découverte des préférences spectateurs par genre :**
   - Les utilisateurs aiment les **films dramatiques** qu’ils prévoient de regarder plus tard (`In Netflix queue`) — ce tag peut refléter de l'intérêt ou de la recommandation indirecte.
   - Les **Thrillers** avec des `twist endings` ou une ambiance `suspense` sont particulièrement appréciés → à prioriser pour la recommandation.

2. **Utilité pour un moteur de recommandation :**
   - En analysant les tags les plus associés à des films bien notés dans chaque genre, on peut mieux orienter les recommandations personnalisées.
   - Par exemple, recommander des **films “Mystery” avec des twists** à ceux qui aiment les “Thrillers” bien notés avec ce tag.

3. **Insights marketing / catégorisation :**
   - Les plateformes peuvent créer des catégories comme :
     - “**Mystery with a Twist**”
     - “**Atmospheric Dramas**”
     - “**Suspenseful Thrillers**”
   - Ces regroupements peuvent améliorer l’engagement en renforçant la correspondance entre ce que les gens aiment et ce qu’on leur propose.

---

### À noter :
- Ce tableau est basé **uniquement sur les films bien notés**, ce qui biaise volontairement l’analyse pour extraire **ce que les utilisateurs apprécient**.
- Les tags sont **libres**, donc il peut y avoir du bruit (noms d’acteurs, fautes de frappe, etc.).
- Une étape de **nettoyage/normalisation des tags** pourrait encore améliorer l’analyse.

---

On peut aussi explorer les **tags associés aux films mal notés**. On pourrait comparer les deux profils de tags pour voir ce qui plaît ou non dans chaque genre.